In [ ]:
import pandas as pd
import numpy as np
import time
import datetime
timestamp_exec_start = time.time()

files = []
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if (filename[-3:] == 'png'):
            files.append(os.path.join(dirname, filename))
print("Processed ",len(files),"files")

labels_dict = {"B_A-":0,"B_F-":1,"B_TA":2,"B_PT":3,"M_DC":4,"M_LC":5,"M_MC":6,"M_PC":7}  # for 8 class problem
labels_dict_simple = {"B":0,"M":1}                                                       # for 2 class problem
REDUCED_CLASSES = False

X = []
Y = []
M = []
for f in files:
    x = f.split("/") # break up the path
    x = x[-1:][0]    # extract the file name
    X.append(str(f))
    if REDUCED_CLASSES:
        Y.append(int(labels_dict_simple[x[4]]))
    else:
        Y.append(int(labels_dict[x[4:8]]))
    m = f.split("-")
    m = m[-2]
    M.append(m)

data = {"images":X,"labels":Y,"magnification":M }
images_df = pd.DataFrame(data, columns = ['images','labels','magnification'])
images_df_40 = images_df.loc[images_df['magnification']=='40'].drop(['magnification'], axis=1)
images_df_100 = images_df.loc[images_df['magnification']=='100'].drop(['magnification'], axis=1)
images_df_200 = images_df.loc[images_df['magnification']=='200'].drop(['magnification'], axis=1)
images_df_400 = images_df.loc[images_df['magnification']=='400'].drop(['magnification'], axis=1)

In [ ]:
# DELETE AFTER APPENDING TO NB ############################################################
batch_size = 16

import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, Dataset

class MyDataset(Dataset):
    def __init__(self, df_data,transform=None):
        super().__init__()
        self.df = df_data.values
        
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path,label = self.df[index]
        
        image = cv2.imread(img_path)
        image = cv2.resize(image, (224,224))
        if self.transform is not None:
            image = self.transform(image)
        return image, label
    
trans_valid = transforms.Compose([transforms.ToPILImage(),                    
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.Resize(224, interpolation = 2),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

##############################################################################################

dataset_img40 = MyDataset(df_data=images_df_40, transform=trans_valid)
loader_img40 = DataLoader(dataset = dataset_img40, batch_size=batch_size//2, shuffle=False, num_workers=0)

dataset_img100 = MyDataset(df_data=images_df_100, transform=trans_valid)
loader_img100 = DataLoader(dataset = dataset_img100, batch_size=batch_size//2, shuffle=False, num_workers=0)

dataset_img200 = MyDataset(df_data=images_df_200, transform=trans_valid)
loader_img200 = DataLoader(dataset = dataset_img200, batch_size=batch_size//2, shuffle=False, num_workers=0)

dataset_img400 = MyDataset(df_data=images_df_400, transform=trans_valid)
loader_img400 = DataLoader(dataset = dataset_img400, batch_size=batch_size//2, shuffle=False, num_workers=0)